<a href="https://colab.research.google.com/github/ArmanBehkish/Computational_Intelligence_23/blob/main/lab3/Nim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Policy Search**
<br> Task: Write agents able to play Nim, with an arbitrary number of rows and an upper bound  on the number of objects that can be removed in a turn (a.k.a., subtraction game).

The player **taking the last object wins.**

>* Task3.1: An agent using fixed rules based on nim-sum (i.e., an expert system)<br>
>* Task3.2: An agent using evolved rules<br>
>* Task3.3: An agent using minmax<br>
>* Task3.4: An agent using reinforcement learning<br>

In [1]:
import logging
import random
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import re
import numpy as np


In [2]:
Nimply = namedtuple("Nimply", "row, num_obj")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2 + 1 for i in range(num_rows)]
        self._k = k

    def nimming(self, ply:Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        # if sum(self._rows) == 0:
        #     logging.debug("Yeuch")
    
    @property
    def rows(self) -> list:
        return self._rows

    def __repr__(self) -> str:
       return(f"Current Game is: {self._rows}")

    def finito(self) -> bool:
      return(sum(self._rows) == 0)

Dictionary to prepare structures for later use.

In [3]:
def struct(GAME) -> dict:
    
    cooked = dict()
    #cooked["possible_moves"] = [(r, o) for r, c in enumerate(GAME.rows) for o in range(1, c + 1) if GAME.k is None or o <= GAME.k]
    cooked["possible_rows"] = [x for x in enumerate(GAME.rows) if x[1]>0]
    cooked["total_active_rows"] = sum(r > 0 for r in GAME.rows)
    cooked["total_one_rows"] = sum(r == 1 for r in GAME.rows)
    cooked["active_rows"] = [r for r in GAME.rows if r>0]
    cooked["shortest_row_idx"] = min((x for x in enumerate(GAME.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row_idx"] = max((x for x in enumerate(GAME.rows)), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(GAME.rows)), key=lambda y: y[1])[1]
    cooked["one_rows_idx"] = [x[0] for x in enumerate(GAME.rows) if x[1] == 1]
    cooked["sorted_rows_from_longest"] = sorted((x for x in enumerate(GAME.rows)),reverse=True, key=lambda y: y[1])

    return cooked


### **Setup of Basic game strategies**
<br> 
1.   Manual Play <br>
2.   Pure Random Play <br>
3.   fix rule Play:  It starts from the longest row and removes all obj except one (systematically conversts all rows to 1). when it reaches a point that all other rows (except the current one) are one, then:


*   if the remmaining ones are even, remove all sticks in the current row -> so the opponent face even number of ones which leave us with the last one : we win!
*    if the remmaining ones are odd, make current row one, and repeat 
*    in case a move is not generated in degenerate situations, random move is done.

> **This strategy always wins the random play, but loses to nim sum!**

     
4.   Nim sum play: (performs the first random move which makes the num-sum of the board zero.) The player playing with this strategy always win.

5.   Evolvable play: plays based on passed genome, used in GA

In [6]:
def manual_play(GAME,player):
  """
    gets the move from input, in the form of row index [space] number of objects
  """
  print(f"Player {player} turn: row,num_objects: ")
  row,num_objs = re.split("\s+",input())
  ply = Nimply(int(row),int(num_objs))
  logging.info(f"player {player} is doing the move: {row},{num_objs}")
  GAME.nimming(ply)
  logging.info(GAME)

def fixrule_play(GAME,player):
  """
    Hard coded rules are as follows:
      It starts from the longest row and removes all obj except one (systematically conversts all rows to 1)
      when it reaches a point that all other rows (except the current one) are one, does this:
        if the remmaining ones are even, remove all sticks in the current row -> so the opponent face even number of ones which leave us with the last one : we win!
        if the remmaining ones are odd, make current row one, and repeat
      in case a move is not generated in degenerate situations, random move is done.

      this strategy wins the random strategy more than 90% of times, but always loses to num_sum as expected
      result of 10,000 games against random:
        Player A has won 9252 games, Player B has won 748 games!
  """
  cooked = struct(GAME)
  row = -1
  num_objs = -1

  if cooked["longest_row"] == 1:
    row = cooked["longest_row_idx"]
    num_objs == 1
  elif cooked["longest_row"] > 1:
    if cooked["total_active_rows"] > cooked["total_one_rows"] + 1:
      row = cooked["longest_row_idx"]
      num_objs = cooked["longest_row"] - 1
    elif cooked["total_active_rows"] == cooked["total_one_rows"] + 1: 
      if cooked["total_one_rows"] % 2 == 0:
        row = cooked["longest_row_idx"]
        num_objs = cooked["longest_row"]
      else:
        row = cooked["longest_row_idx"]
        num_objs = cooked["longest_row"] - 1

  assert GAME.rows[row] >= num_objs
  if row is -1 or num_objs == -1:
    logging.warning(f"playing random from hardcode, values are {row},{num_objs}")
    random_play(GAME,player)
    return None
  logging.info(f"player {player} is doing the move: {row},{num_objs}")
  ply = Nimply(row,num_objs)
  GAME.nimming(ply)
  logging.info(GAME)


def random_play(GAME,player):
  """
    Pure random choose of row and number of objects to remove
  """
  cooked = struct(GAME)
  row_num,total_num_objs = cooked["possible_rows"][random.randint(0,len(cooked["possible_rows"])-1)]
  num_obj = random.randint(1,total_num_objs)
  logging.info(f"player {player} is doing the move: {row_num},{num_obj}")
  ply = Nimply(row_num,num_obj)
  GAME.nimming(ply)
  logging.info(GAME)


def nimsum_play(GAME: Nim,player):
  """
    chooses move that makes the numsum zero, when such move cannot be found, rplays randomly
  """
  assert sum(GAME._rows) > 0
  iamstuckcounter = 0
  while(True):
    row = random.randint(0,len(GAME._rows)-1)
    num_obj = random.randint(0,GAME._rows[row])
    if GAME._rows[row] is 0 or num_obj is 0:
      continue
    temp_game_rows = deepcopy(GAME.rows)
    temp_game_rows[row] -= num_obj
    *_, result = accumulate(temp_game_rows, xor)
    if result==0:
      logging.info(f"player {player} is doing the move: {row},{num_obj}")
      ply = Nimply(row,num_obj)
      GAME.nimming(ply)
      logging.info(GAME)
      break
    else:
      iamstuckcounter +=1
      if iamstuckcounter > 201:
        logging.warning(f"player {player} cannot find a nimsum move, going random....{row,num_obj}.")
        ply = Nimply(row,num_obj)
        GAME.nimming(ply)
        logging.info(GAME)
        break
      continue  

def evolvable_play(GAME,player,genome=(86,99)):
  """
    genome = (n,p)
    used for genetic algorithm, palys based on passed genome
    It sorts active rows based on length and then chooses based on percentile given as n
    removes obejects based on percentile of objects in the selected row, p
  """
  row = -1
  num_obj = -1
  n = genome[0]
  p = genome[1]
  cooked = struct(GAME)
  length = cooked["total_active_rows"] 
  row,total_num_objs = [x for x in cooked['sorted_rows_from_longest'] if x[1] > 0][np.percentile(range(length),n,interpolation='nearest')]
  num_obj = np.percentile([x for x in range(total_num_objs+1) if x > 0],p,interpolation='nearest')
  if row is -1 or num_obj is -1:
    logging.warning(f"playing random from evolvable, values are {row},{num_obj}")
    random_play(GAME,player)
    return None
  logging.info(f"player {player} is doing the move: {row},{num_obj}")
  ply = Nimply(row,num_obj)
  GAME.nimming(ply)
  logging.info(GAME)
  return None


def play(game_size,N, a_strategy, b_strategy,players,genome=None):
  """
    plays N games between 2 given strategies and return number of wins
  """
  a_wins = 0
  b_wins = 0
  opponent = (a_strategy, b_strategy)
  for i in range(N):
    GAME = Nim(game_size)
    logging.info(f"{GAME}, players: {players}")
    # Main game loop
    while(True):
      player=0
      if a_strategy is evolvable_play and genome is not None:
        opponent[player](GAME,players[player],genome)
      else:
        opponent[player](GAME,players[player])
      if(GAME.finito()):
        logging.warning(f"Winner is Player: {players[player]}!")
        a_wins += 1
        break  
      player=1
      if b_strategy is evolvable_play and genome is not None:
        opponent[player](GAME,players[player],genome)
      else:
        opponent[player](GAME,players[player])
      if(GAME.finito()):
        logging.warning(f"Winner is Player: {players[player]}!")
        b_wins += 1
        break
  return (a_wins,b_wins)


# GENETIC ALGORITHM



> The genome is defined as 2 parameters: 


>> parameter to select row, from 0(longest row) to 100(shorest row) <br>
>> parameter to remove objects, between 0(remove one) to 100(remove all) <br>

> Two kinds of mutations are used, one each element of genome by adding/subtracting a given purturbation.

> fintess is defined by the percentage of wins against a strategy

> It tries to compete against random until a level(85%) of fitness, then goes to fixed rule and then nim sum, so as to provide a gradual hill to climb. Although it is doubtful how much this affects the outcome!

**The learned player, although with 2 simple parameters, maneges to win over 90% of times against random player, but loses completely to nimsum!**













In [5]:
# Evolution
# n = 65          # parameter to select row, from 0(longest row) to 100(shorest row)
# p = 80         # parameter to remove objects, between 0(remove one) to 100(remove all)
# genome = (n,p)

game_size = 10

POPULATION_SIZE = 50
OFFSPRING_SIZE = 30
NUM_GENERATIONS = 30
row_mutation_prob = 0.55
#obj_mutation_prob = 0.45

Individual = namedtuple("Individual", ["genome", "fitness"])

strategies = (random_play,fixrule_play,nimsum_play)
fitness_number_plays = 100
fitness_criteria = (strategies[0],fitness_number_plays)      # opponent play strategy, number of plays
fitness_log = list()

def fitness(genome,game_size,fintess_criteria):
  """
    calculates the fitness of an individual as percentage win while playing against an opponent strategy
  """
  opp_strategy = fitness_criteria[0]
  num_play = fitness_criteria[1]

  players = {0:"A",1:"B"}
  a_strategy = evolvable_play
  b_strategy = opp_strategy
  num_wins, _ = play(game_size,num_play, a_strategy, b_strategy,players,genome=genome)
  return round((num_wins/num_play)*100)

def tournament(population, tournament_size=2):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)
  
def mutate_row(genome):
  """
    mutate row number by randomly adding or removing perturbation from it
  """
  n = genome[0]
  p = genome[1]
  perturbation_size = 5
  operator_functions = {
  '+': lambda a, b: a + b, 
  '-': lambda a, b: a - b,
  }
  perturbation = random.randint(0,perturbation_size)
  new_n = operator_functions[random.choice(['+','-'])](n, perturbation)
  if new_n > 100:
    new_n = 100
  if new_n < 0:
    new_n = 0
  return (new_n,p)

def mutate_objs(genome):
  """
    mutate object number by randomly adding or removing perturbation from it
  """
  n = genome[0]
  p = genome[1]
  perturbation_size = 2
  operator_functions = {
  '+': lambda a, b: a + b, 
  '-': lambda a, b: a - b,
  }
  perturbation = random.randint(0,perturbation_size)
  new_p = operator_functions[random.choice(['+','-'])](p, perturbation)
  if new_p > 100:
    new_p = 100
  if new_p < 0:
    new_p = 0
  return (n,new_p)

#Initial population
population = list()
for genome in [(random.randint(0,100),random.randint(0,100)) for _ in range(POPULATION_SIZE)]:
    population.append(Individual(genome,fitness(genome,game_size,fitness_criteria)))

fitness_log = [(0, i.fitness) for i in population]

# Evolution Loop
for generation in range(NUM_GENERATIONS):
    offspring = list()
    for offs in range(OFFSPRING_SIZE):
        if random.random() < row_mutation_prob:
            i = tournament(population)
            ng = mutate_row(i.genome)
        else:
            i = tournament(population)
            ng = mutate_objs(i.genome) 
        highest_fitness = sorted(fitness_log,reverse=True,key=lambda x:x[1])[0][1]
        if highest_fitness > 95:
          f = fitness(ng,game_size,(strategies[2],fitness_number_plays))
        elif highest_fitness > 85:
          f = fitness(ng,game_size,(strategies[1],fitness_number_plays))
        else:
          f = fitness(ng,game_size,(strategies[0],fitness_number_plays))
        fitness_log.append((generation+1, f))
        offspring.append(Individual(ng, f))
    population += offspring
    population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]


print(population)


Streaming output truncated to the last 5000 lines.


[Individual(genome=(86, 99), fitness=100), Individual(genome=(76, 100), fitness=99), Individual(genome=(87, 98), fitness=99), Individual(genome=(87, 98), fitness=99), Individual(genome=(85, 98), fitness=99), Individual(genome=(84, 98), fitness=99), Individual(genome=(76, 100), fitness=99), Individual(genome=(83, 99), fitness=98), Individual(genome=(82, 99), fitness=98), Individual(genome=(75, 100), fitness=98), Individual(genome=(77, 99), fitness=98), Individual(genome=(79, 99), fitness=98), Individual(genome=(85, 98), fitness=98), Individual(genome=(87, 98), fitness=98), Individual(genome=(86, 98), fitness=98), Individual(genome=(92, 98), fitness=98), Individual(genome=(85, 98), fitness=98), Individual(genome=(87, 99), fitness=98), Individual(genome=(79, 100), fitness=97), Individual(genome=(87, 98), fitness=97), Individual(genome=(87, 99), fitness=97), Individual(genome=(76, 99), fitness=97), Individual(genome=(78, 98), fitness=97), Individual(genome=(81, 99), fitness=97), Individual

### Throw the game N times to evaluate the performance of strategies

Different Strategies could be selected for test purposes:


*   Manual Play
*   Purely Random Play
*   Nimsum Play
*   fix-rule play
*   evolvable play (can play based on given genome in the form (p,q). could be used to test the result of genetic algorithm

The result of N plays is reported!



In [16]:
logging.getLogger().setLevel(logging.ERROR)
#Number of Plays
N = 100
#Confiture Game Board
game_size = 10
players = {0:"A",1:"B"}
#Set strategy for each player; manual_play; random_play; nimsum_play; fixrule_play; evolvable_play
a_strategy = evolvable_play
b_strategy = random_play

a_wins, b_wins = play(game_size,N, a_strategy, b_strategy,players)

print(f"Player {players[0]} has won {a_wins} games, Player {players[1]} has won {b_wins} games!")

Player A has won 97 games, Player B has won 3 games!
